In [30]:
#pip install httplib2==0.12.0

     |████████████████████████████████| 218 kB 8.5 MB/s eta 0:00:01
  Created wheel for httplib2: filename=httplib2-0.12.0-py3-none-any.whl size=93465 sha256=fa73c0a659f4e63188a03a081a9a2246b0c732d8b55c22a78c1ed06076c0e6c4
  Stored in directory: /root/.cache/pip/wheels/0d/e7/b6/0dd30343ceca921cfbd91f355041bd9c69e0f40b49f25b7b8a
Successfully built httplib2
ERROR: google-auth-httplib2 0.1.0 has requirement httplib2>=0.15.0, but you'll have httplib2 0.12.0 which is incompatible.
ERROR: google-api-python-client 2.3.0 has requirement httplib2<1dev,>=0.15.0, but you'll have httplib2 0.12.0 which is incompatible.
  Attempting uninstall: httplib2
    Found existing installation: httplib2 0.19.1
    Uninstalling httplib2-0.19.1:
      Successfully uninstalled httplib2-0.19.1
You should consider upgrading via the '/root/apache-beam-2.25.0/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
#pip freeze

absl-py==0.13.0
# Editable install with no version control (apache-beam==2.25.0)
-e /root/apache-beam-2.25.0/packages/beam/sdks/python
argon2-cffi==20.1.0
astunparse==1.6.3
async-generator==1.10
attrs==21.2.0
avro-python3==1.8.2
backcall==0.2.0
bleach==3.3.1
cachetools==4.2.2
certifi==2021.5.30
cffi==1.14.6
charset-normalizer==2.0.3
crcmod==1.7
cycler==0.10.0
Cython==0.29.13
debugpy==1.3.0
decorator==5.0.9
defusedxml==0.7.1
dill==0.3.1.1
docopt==0.6.2
entrypoints==0.3
facets-overview==1.0.0
fastavro==1.0.0.post1
fasteners==0.16.3
future==0.18.2
gast==0.3.3
google-api-core==1.22.0
google-apitools==0.5.31
google-auth==1.33.1
google-auth-oauthlib==0.4.4
google-cloud-bigquery==1.26.1
google-cloud-bigtable==1.0.0
google-cloud-build==2.0.0
google-cloud-core==1.4.1
google-cloud-datastore==1.7.4
google-cloud-dlp==1.0.0
google-cloud-language==1.3.0
google-cloud-pubsub==1.0.2
google-cloud-spanner==1.19.1
google-cloud-videointelligence==1.16.1
google-cloud-vision==1.0.0
google-crc32c==1.1.2
googl

In [1]:
import logging
import re
import json
import time
import traceback
import google.auth

import apache_beam as beam
from apache_beam import WithKeys, GroupByKey
from apache_beam import FlatMap, Map, ParDo, Flatten, Filter, GroupBy
from apache_beam import Values, CombineGlobally, CombinePerKey
from apache_beam import pvalue, window, WindowInto
from apache_beam.options import pipeline_options
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.transforms.combiners import Top, Mean, Count
from apache_beam.io.textio import ReadFromText, WriteToText
from apache_beam.io.gcp.pubsub import ReadFromPubSub
from apache_beam.io.gcp.bigquery import BigQueryDisposition, WriteToBigQuery

from apache_beam.runners import DataflowRunner
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner
import apache_beam.runners.interactive.interactive_beam as ib

from apache_beam.testing.util import assert_that, is_empty, equal_to

from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import SetupOptions
from apache_beam.options.pipeline_options import GoogleCloudOptions
from apache_beam.options.pipeline_options import StandardOptions

In [11]:
project = google.auth.default()[1]
bucket = 'gs://' + 'temp_crypto_batch_' + google.auth.default()[1].replace('-','_')
bucket

'gs://temp_crypto_batch_verdant_cargo_321713'

In [3]:
logging.getLogger().setLevel(logging.INFO)

In [4]:
from setuptools import setup, find_packages

In [6]:
%%writefile setup.py

from setuptools import setup, find_packages

setup(
    name="dataflow_pipeline_dependencies",
    version="0.1",
    install_requires=[
   'numpy==1.17.3', 'pandas==1.2.4', 'pystan==2.19.1.1', 'prophet==1.0.1', 'pytz==2019.3'
    ],
    packages = find_packages()
)

Overwriting setup.py


In [7]:
query = """
SELECT
  *
FROM
  `temp_crypto_batch.batch_hour_interpo`
WHERE 
  tumble <= (SELECT TIMESTAMP_SUB(max(tumble), INTERVAL 5 DAY) FROM `temp_crypto_batch.batch_hour_interpo`)
"""

In [12]:
pipeline_parameters = [
    '--project', project,
    '--staging_location', "%s/staging_bq_notebook" % bucket,
    '--temp_location', "%s/temp_bq_notebook" % bucket,
    "--setup_file", './setup.py', 
    "--region", "us-central1",
    "--template_location", bucket + '/prediction/templates/CUSTOM_TEMPLATE_DATAFLOW'
]

In [13]:
class KeepDoFn(beam.DoFn):
    def process(self, element):
        return [{
            'symbol': element['symbol'],
            'ds': element['tumble'],
            'y' : element['intrp']
    }]

###########################
    
class KeepDoFnAfterGrouping(beam.DoFn):
    def process(self, element):
        return [{
            'symbol': element[0],
            'data': element[1]
    }]
    
###########################

class Prophet_M(beam.DoFn):
    def process(self, element):
        import pandas
        #from datetime import datetime
        import numpy as np
        from prophet import Prophet
        
        symbol = element['symbol']
        data = element['data']
        
        data = pandas.DataFrame(data)
        data['ds'] = data['ds'].dt.tz_localize(None)
        
        ##############
        
        max_datetime = max(data['ds'])
        # Extract training data
        train_data = data[data['ds'] <= max_datetime]
        
        #############
        
        m = Prophet(seasonality_mode="multiplicative") 
            
        #############
        
        """Fit the model using the training data."""
        m.fit(train_data)
        
        ############
        
        future = m.make_future_dataframe(periods=120, freq='H')      
        forecast = m.predict(future)
        
        ###########
        
        forecast=pandas.merge(forecast, train_data, on='ds', how='left')
        
        x = np.array(symbol)
        forecast['symbol']=np.repeat(x, len(forecast), axis=0)
        
        ############
        
        forecast_dict = forecast.to_dict('records')
        
        return [{
        'symbol' : symbol,
        'forecast': forecast_dict   
        }]
    
########################

class Expand_Predictions_M(beam.DoFn):
    def process(self, element):
        import pandas, pytz, math
        
        return [{
        'symbol' : element['symbol'],
        'ds': pytz.utc.localize(element['ds']).to_pydatetime(),
        'y': element['y'] if not(math.isnan(element['y'])) else None,
        'yhat': element['yhat'],
        'yhat_lower': element['yhat_lower'], 
        'yhat_upper': element['yhat_upper']
        }]
    
#######################

table = "{}:temp_crypto_batch.test_prophet_global".format(project)
schema = "symbol:STRING, ds:TIMESTAMP, y:FLOAT, yhat:FLOAT, yhat_lower:FLOAT, yhat_upper:FLOAT"

p = beam.Pipeline("DataFlowRunner", argv=pipeline_parameters)
create = (p | "Create" >>  beam.io.ReadFromBigQuery(query=query,use_standard_sql=True)
            | "Map Keys" >> beam.ParDo(KeepDoFn())
            | "Add Keys" >> WithKeys(lambda x: x["symbol"])
            | GroupByKey()
            | "Rename After Grouping" >> beam.ParDo(KeepDoFnAfterGrouping())
            | "Prophet Data" >> beam.ParDo(Prophet_M())
            | "Flat Map Data" >> FlatMap(lambda x: x["forecast"])
            | "Expand Predictions" >> beam.ParDo(Expand_Predictions_M())
            | "Write To BigQuery" >> WriteToBigQuery(table=table, schema=schema,
                                  create_disposition=BigQueryDisposition.CREATE_IF_NEEDED,
                                  write_disposition=BigQueryDisposition.WRITE_TRUNCATE))

p.run()

/root/apache-beam-2.25.0/packages/beam/sdks/python/apache_beam/io/gcp/bigquery.py:1971: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  temp_location = pcoll.pipeline.options.view_as(
/root/apache-beam-2.25.0/packages/beam/sdks/python/apache_beam/io/gcp/bigquery_file_loads.py:900: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  temp_location = p.options.view_as(GoogleCloudOptions).temp_location
INFO:apache_beam.runners.portability.stager:Executing command: ['/root/apache-beam-2.25.0/bin/python', 'setup.py', 'sdist', '--dist-dir', '/tmp/tmp5xz3nbyi']
INFO:apache_beam.runners.portability.stager:Downloading source distribution of the SDK from PyPi
INFO:apache_beam.runners.portability.stager:Executing command: ['/root/apache-beam-2.25.0/bin/python', '-m', 'pip', 'download', '--dest', '/tmp/tmp5xz3nbyi', 'apache-beam==2.25.0', 

<DataflowPipelineResult None at 0x7f670987f710>